# 댓글 데이터 분석
* 데이터 모으기
* 데이터 전처리
* 데이터 시각화

In [1]:
%config Completer.use_jedi = False


import sys
sys.getdefaultencoding()

import re
import requests
from datetime import datetime
import pandas as pd
import collections as co

client_id = "OP08QraFd_zLrAfjBrTA"  
client_secret = "qcS0nZixH7"



## Date

In [2]:
start_date = datetime(2021,11,17)
end_date = datetime(2021,11, 24)

In [3]:
keyword = "김동연"
respond_type = "json"
Ndisplay = 100
sort_type = 'sim'

headers = {
    'X-Naver-Client-Id' : client_id, 
    'X-Naver-Client-Secret':client_secret

}


res = pd.DataFrame({ 'title': [None], 'originallink': [None], 'link': [None], 'description': [None], 'pubDate': [None] }) 

for start in range(1,1001, 100):
    search_word = re.sub('[^0-9a-zA-Zㄱ-힗]', ' ', keyword) 
    url = f"https://openapi.naver.com/v1/search/news.{respond_type}?query={keyword}&display={str(int(Ndisplay))}&start={str(int(start))}&sort={sort_type}"
    r = requests.get(url, headers=headers) 
    print(r.status_code)
    if r.status_code == 200: 

        temp = pd.DataFrame(r.json()['items']) 

    else: 
        print('error with errorCode:' + r.status_code)

    res = res.append(temp)

200
200
200
200
200
200
200
200
200
200


In [4]:
res = res.dropna()

res['pubDate'] = pd.to_datetime(res['pubDate'])
res['pubDate'].loc[0].tolist()[0].strftime("%Y-%m-%d")

res['pubDate'] =  res['pubDate'].map(lambda x : x.strftime("%Y-%m-%d"))
res['pubDate'] = pd.to_datetime(res['pubDate'])

res = res[(res['pubDate'] > start_date) & (res['pubDate'] < end_date)]

res.to_csv("./sns_data/{}/article/article.csv".format(end_date.strftime("%Y-%m-%d"))
           , encoding='utf-8')

## 댓글 가져오기
* https://signing.tistory.com/77

In [5]:
from selenium import webdriver 
from selenium import webdriver as wd
 
from selenium.webdriver.chrome.options import Options

from datetime import datetime
import pandas as pd
import numpy as np
import os
import collections as co

import time

url = 'https://news.naver.com/main/read.naver?mode=LSD&mid=shm&sid1=102&oid=079&aid=0003561032'

driverPath = "/usr/bin/chromedriver"

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(driverPath,chrome_options=chrome_options)
driver.implicitly_wait(30)
driver.get(url)

#더보기 계속 클릭하기
while True:
    try:
        더보기 = driver.find_element_by_css_selector('a.u_cbox_btn_more')
        더보기.click()
        time.sleep(1)
    except:
        break

#댓글추출
contents = driver.find_elements_by_css_selector('span.u_cbox_contents')
for content in contents:
    print(content.text)

/home/nakyil/anaconda3/envs/DLassn/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: use options instead of chrome_options


WebDriverException: Message: 'chromedrier' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [5]:
# url = res['link'].iloc[0]
# driver.get(url)

# #더보기 계속 클릭하기
# while True:
#     try:
#         더보기 = driver.find_element_by_css_selector('a.u_cbox_btn_more')
#         더보기.click()
#         time.sleep(1)
#     except:
#         break

# #댓글추출
# contents = driver.find_elements_by_css_selector('span.u_cbox_contents')
# for content in contents:
#     print(content.text)


In [6]:
#Back-up 
def backup(i, article, reples): 
    if reples:
        if (int(i)-1) % 10 == 0: 
            # free article body 
            res_article = pd.DataFrame(article) 
            res_article.to_csv('./NaverNews_body_reple/NaverNews_Exist_article'+str(i)+'.csv', index=False, encoding='utf-8-sig') 
            del res_article, article 
            log.write(message+'Delete res, article and free memory') 
            log.write(message+'Re-create article') 
            article = co.deque([]) 
            # free article reple 
            res_reple = pd.concat(reples) 
            res_reple.to_csv('./NaverNews_body_reple/NaverNews_Exist_reple_'+str(i)+'.csv', index=False, encoding='utf-8-sig') 
            del res_reple, reples 
            log.write(message+'Delete res, article and free memory') 
            log.write(message+'Re-create article') 
            reples = co.deque([])



In [7]:
def enter_URL(url): 

    #import ipdb; ipdb.set_trace(context=15)
    try: 
#             driver = wd.Chrome(executable_path=driverPath) 
#             driver.get(url) 

        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome(driverPath,chrome_options=chrome_options)
        driver.implicitly_wait(30)
        driver.get(url)
        return True
        
        
    except:
        print("error")
        log.write(message+'error with get URL... maybe Blocked IP... sleep 10 sec...') 
        log.write(message+'restart') 

        return False



In [8]:
# Get data - Article body 
def Get_Article_Body(i, article, url): 
    
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(driverPath,chrome_options=chrome_options)
    driver.implicitly_wait(30)
    driver.get(url)
    body = driver.find_elements_by_xpath("//div[@id='articleBodyContents']") 
    print("temp")
    text = "" 
    for b in body: 
        text = text.join(b.text) 
    
    # re-enter 
    if text.strip(url) == '': 
        log.write(message+'there`s no article... maybe loading error') 
        driver.refresh() 
        log.write(message+'refresh...') 
        log.write(message+'sleep 5 sec...') 
        time.sleep(5) 
        log.write(message+'try again...') 
        driver.get(url) 
        body = driver.find_elements_by_xpath("//div[@id='articleBody']") 
        text = "" 
        for b in body: 
            text = text.join(b.text) 
            
        tmp_article = { 'NaverURL':url, 'body':text } 
        article.append(tmp_article)
    
    return article

In [9]:
# Get data - Article reple 
def Get_Ariticle_reple(i, reples, record): 
    
        
    title = record[1]['title']
    pubDate = record[1]['pubDate']
    description = record[1]['description']
    
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(driverPath,chrome_options=chrome_options)
    driver.implicitly_wait(30)
    driver.get(url)
    log.write(message+'sleep 2 sec for scrap reples...') 
    time.sleep(2) 
    noReple = False 
    #no_reple = pd.DataFrame({ 'repleID':[None], 'reple':[None], 'time':[None], 're_reple':[None], 'like':[None], 'dislike':[None], 'NaverURL':[None] }) 
    no_reple = pd.DataFrame({}) 
    try: # Click more reple 
        driver.find_elements_by_xpath("//span[@class='u_cbox_in_view_comment']")[0].click() 
    except: 
        print("No Reple")
        noReple = True 
    
    if noReple: 
        tmp_reple = no_reple 
    else: 
        log.write(message+'there are some reples... refresh and sleep 2 sec to scrap all data...') 
        driver.refresh() 
        time.sleep(2) # 총 댓글 갯수 확인 
    
    if noReple:
        pass
    else :    
        Nreple = driver.find_element_by_xpath("//span[@class='u_cbox_count']") 
        Nreple = Nreple.text 
        Nreple = int(Nreple.replace(',','')) 
        print(Nreple) 
        if Nreple > 20: 
            NrepleLoop = (Nreple//20) if Nreple%20 > 0 else (Nreple//20)-1 
            for i in range(NrepleLoop): 
                driver.find_elements_by_xpath("//span[@class='u_cbox_page_more']")[0].click() 
                time.sleep(1) 

    # 댓글 수집 
    contents = []
    try :
        contents = driver.find_elements_by_css_selector('span.u_cbox_contents')
    
    except :
        print("Error")
    
    tmp_reple = pd.DataFrame({})
    if len(contents) > 0:
        tmp_reple = co.deque([])
        for content in contents:
            print("content", content.text)
            tmp_list = content.text.split("\n")
            tmp_reple.append(tmp_list)
        tmp_reple = pd.DataFrame(tmp_reple)
        tmp_reple['NaverURL'] = url
        tmp_reple['pubDate'] = pubDate
        tmp_reple['description'] = description
                


#     reple = driver.find_elements_by_xpath('//div[@class="u_cbox_comment_box"]') 
    
#     if len(reple) > 0:
#         # 댓글이 적어도 하나라도 있는 경우: 클린봇 X, 삭제 X 
#         tmp_reple = co.deque([]) 
#         for r in reple: 
#             tmp_list = r.text.split('\n') 
#             if len(tmp_list) == 11: 
#                 tmp_reple.append(tmp_list) 
#         tmp_reple = pd.DataFrame(tmp_reple) 
#         if len(tmp_reple) == 0: 
#             # 댓글 하나만 있는데 삭제되었을 때 
#             tmp_reple = no_reple 
#         else: 
            
#             tmp_reple = tmp_reple[[0,3,4,5,8,10]].rename(columns={ 0:'repleID', 3:'reple', 4:'time', 5:'re_reple', 8:'like', 10:'dislike' }) 
#             tmp_reple['NaverURL'] = url 
            
#     else: 
#         tmp_reple = no_reple 
    reples.append(tmp_reple)
    return reples

In [10]:
res = pd.read_csv("result.csv", encoding="utf-8")
res = res.dropna()
res['link']

0      https://news.naver.com/main/read.naver?mode=LS...
1      https://news.naver.com/main/read.naver?mode=LS...
2      https://news.naver.com/main/read.naver?mode=LS...
3      https://news.naver.com/main/read.naver?mode=LS...
4      http://www.gukjenews.com/news/articleView.html...
                             ...                        
991    https://news.naver.com/main/read.naver?mode=LS...
992    https://news.naver.com/main/read.naver?mode=LS...
993    https://news.naver.com/main/read.naver?mode=LS...
994    https://news.naver.com/main/read.naver?mode=LS...
995    http://www.newsway.co.kr/news/view?tp=1&ud=202...
Name: link, Length: 996, dtype: object

In [11]:
res.head()

,Unnamed: 0,Unnamed: 0.1,title,originallink,link,description,pubDate
0,1,0,"[여의도 사사건건] <b>김동연</b> “손실보상 많이 미흡…어설픈 지원, 재정만 ...",https://news.kbs.co.kr/news/view.do?ncd=529682...,https://news.naver.com/main/read.naver?mode=LS...,출연 : <b>김동연</b> 대선 예비 후보 https://youtu.be/jc0t...,"Fri, 08 Oct 2021 17:04:00 +0900"
1,2,1,[논담] <b>김동연</b> &quot;'공통공약' 수렴해 누가 대통령 되든 이행 ...,https://hankookilbo.com/News/Read/A20211007152...,https://news.naver.com/main/read.naver?mode=LS...,실용적 경세가로서 제3지대 독자출마를 선언한 <b>김동연</b> 전 경제부총리는 대...,"Thu, 07 Oct 2021 17:01:00 +0900"
2,3,2,"[정면승부] <b>김동연</b> &quot;재정 건전성보다 회복 탄력성 우선해야, ...",https://www.ytn.co.kr/_ln/0101_202110062026588197,https://news.naver.com/main/read.naver?mode=LS...,5 (17:30~19:30) ■ 방송일 : 2021년 10월 6일 (수요일) ■ 대...,"Wed, 06 Oct 2021 20:26:00 +0900"
3,4,3,"`잠행` <b>김동연</b>, 창준위 발족 초읽기…`제3지대` 세력화 시동",http://www.edaily.co.kr/news/newspath.asp?news...,https://news.naver.com/main/read.naver?mode=LS...,차기 대선주자인 <b>김동연</b> 전 경제부총리가 오는 17일 무렵에 정식으로 창...,"Thu, 07 Oct 2021 14:38:00 +0900"
4,5,4,"[특징주]보해양조 주가 10% 상승, <b>김동연</b> 관련주 언급",http://www.gukjenews.com/news/articleView.html...,http://www.gukjenews.com/news/articleView.html...,"보해양조 주가 10% 상승, <b>김동연</b> 관련주 언급(사진=보해양조 CI) ...","Fri, 08 Oct 2021 15:12:00 +0900"


In [12]:
start = 430
res['index'] = list(range(res.shape[0]))

URLs = res
article = []
reples = []
reples_total = []


log = open('./logs/log_'+datetime.strftime(datetime.now(), "%Y%m%d_%H%M%S")+'.txt', 'w') 
log.write('start program: '+datetime.strftime(datetime.now(), "%H:%M:%S.%f")) 
print('start program: '+datetime.strftime(datetime.now(), "%H:%M:%S.%f"))

log = open('./logs/log_'+datetime.strftime(datetime.now(), "%Y%m%d_%H%M%S")+'.txt', 'w') 
log.write('start program: '+datetime.strftime(datetime.now(), "%H:%M:%S.%f")) 
print('start program: '+datetime.strftime(datetime.now(), "%H:%M:%S.%f"))

for idx, record in enumerate(URLs.iloc[start:].iterrows()):
    print("======================")
    print("Start ", idx+ start)
    i = record[1]['index']
    url = record[1]['link']

    
    if url == None:
        continue
    message = '\n'+datetime.strftime(datetime.now(), "%H:%M")+' '+str(i)+'th loop | '
# Back-up function # Article body function # Article reple function
    print("Back up")
    backup(i, article, reples)
    print("Enter URL")
    accessed = enter_URL(url)
    
    if accessed :
        print("Get Article Body")
        article = Get_Article_Body(i, article, url)
        print("Get reple")
        reples = Get_Ariticle_reple(i, reples, record)
        reples_total.extend(reples)

        driver.quit() 
        message2 = '\n'+datetime.strftime(datetime.now(), "%H:%M")+' '+str(i)+'th loop | ' 
        log.write(message2+'NaverURL: '+url) 
        print(message2+'NaverURL: '+url)
    
res_article = pd.DataFrame(article)
res_reple = pd.concat(reples)
res_article.to_csv('./NaverNews_body_reple/NaverNews_Exist_article.csv', index=False)
res_reple.to_csv('./NaverNews_body_reple/NaverNews_Exist_reple.csv', index=False)

log.write('\n end program: '+datetime.strftime(datetime.now(b), "%H:%M:%S.%f"))
print('end program')

log.close()

start program: 09:36:45.489136
start program: 09:36:45.489735
Start  430
Back up
Enter URL


/home/nakyil/anaconda3/envs/DLassn/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


Get Article Body


/home/nakyil/anaconda3/envs/DLassn/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


temp
Get reple


/home/nakyil/anaconda3/envs/DLassn/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


No Reple

09:39 430th loop | NaverURL: https://www.dnews.co.kr/uhtml/view.jsp?idxno=202109131305387570410
Start  431
Back up
Enter URL
Get Article Body
temp
Get reple
No Reple

09:40 431th loop | NaverURL: https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=100&oid=119&aid=0002528331
Start  432
Back up
Enter URL
Get Article Body
temp
Get reple
No Reple

09:51 432th loop | NaverURL: https://www.ccdailynews.com/news/articleView.html?idxno=2080747
Start  433
Back up
Enter URL
Get Article Body
temp
Get reple
No Reple

09:53 433th loop | NaverURL: http://www.newstomato.com/ReadNews.aspx?no=1073809&inflow=N
Start  434
Back up
Enter URL
Get Article Body
temp
Get reple
No Reple

09:55 434th loop | NaverURL: http://www.newsway.co.kr/news/view?tp=1&ud=2021091316313142920
Start  435
Back up
Enter URL
Get Article Body
temp
Get reple
No Reple

09:58 435th loop | NaverURL: http://www.kbmaeil.com/news/articleView.html?idxno=908894
Start  436
Back up
Enter URL
Get Article Body
temp
Get reple


TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.532
  (Session info: headless chrome=94.0.4606.81)


In [ ]:
res_reple = pd.concat(reples_total)
res_reple = res_reple.reset_index(drop=True)
res_reple.to_csv('./NaverNews_body_reple/NaverNews_Exist_reple.csv')


In [ ]:
i[1]['index']

In [ ]:
for i in URLs.iterrows():
    print(i)
    title = i[1]['title']
    index = i[1]['index']
    url = i[1]['link']
    pubDate = i[1]['pubDate']
    description = i[1]['description']
    break

In [154]:
i[1]

Unnamed: 0                                                      1
Unnamed: 0.1                                                    0
title           [여의도 사사건건] <b>김동연</b> “손실보상 많이 미흡…어설픈 지원, 재정만 ...
originallink    https://news.kbs.co.kr/news/view.do?ncd=529682...
link            https://news.naver.com/main/read.naver?mode=LS...
description     출연 : <b>김동연</b> 대선 예비 후보 https://youtu.be/jc0t...
pubDate                           Fri, 08 Oct 2021 17:04:00 +0900
index                                                           0
Name: 0, dtype: object